In [ ]:
# run this cell
from datascience import *
import numpy as np

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Pygrowup
We will install a program from the World Health Organization to use to help. Below are two ways this can be done:

1. Click on the Jupyter icon above. Go to new & select terminal. type code `pip install --user pygrowup`, then press enter.
2. Add a new cell within this notebook. type code `!pip install --user pygrowup`, then run the cell.

In [ ]:
from pygrowup import Calculator
calculator = Calculator(adjust_height_data=False, adjust_weight_scores=False,
                       include_cdc=False, logger_name='pygrowup',
                       log_level='INFO')

In [ ]:
data = Table.read_table('Lab_2.csv')
data

# Anthropometric measures
We are going to look at data about growth.

The growth chart is particularly nice because instead of having to know what everyone's height should be at the different ages, we can compare across ages using one number.  Children grow as they get older, but by how much should they be growing?  These growth charts help us compare across ages.

![](Lab_3_Image_1.jpg)

<font color="blue"> Item 1: Using your data, make a scatter plot of height in centimeters and age in months for children under 5 years (60 months).  Do not include missing values (-99).
> Visually inspect the graph:  What is average height/length at 1 month?  How about at 18 months?  At 3 years?  
> How does the spread of heights change as children age?

In [ ]:
# make a scatter plot
### STUDENT ANSWER


![](Lab_3_Image_2.jpg)

Is your graph similar to the above chart?  The above chart shows the ranges of healthly growth.  Similar to percentiles, z-scores are comparable across age.  The z-scores in the chart below were generated based on well-nourished & healthy children all over the world. 

<font color="blue"> Item 2: Calculate average height for girls age 0-2 months, 23-25 months, and 58-60 months.  Using the growth chart above, what are the average girl's z-scores for these ages?  Use the middle month when reading the chart (1 month, 2 years, 5 years).  At which age are children in your population doing the best, relative to healthy children around the world?

In [ ]:
girls = ... # make a table that includes only females
avg1 = ... # average height for girls 0-2 months old
avg2 = ... # average height for girls 23-25 months old
avg3 = ... # average height for girls 58-60 months old
(avg1, avg2, avg3)
### STUDENT ANSWER


The World Health Organization has provided data a program to do these transformations for us. Keep only children under age 5 (60 months or less) and those with an age in months value. Implement the pygrowup code for these children. 

*Note*: Some data sets will have a some sort of combination of a `years old` and `months` column that need to be combined for `months old` column. For example, if `years old` is 3 and `months` is 2, that individual is $ 3 * 12 + 2 = 38 $ months old.

<font color="Blue"> Item 3:  What variables does Pygrowup generate?

In [ ]:
# Use this cell to address missing values (drop them).  
valid_children = ... # table with only valid ages
valid_children.show(100)
### STUDENT ANSWER


In [ ]:
#formula from Pygrowup
def compute_z_score(row):
    if row.item('Sex') == 1:
        sex = 'M'
    else:
        sex = 'F'
    return calculator.lhfa(row.item('Height'), row.item('Months Old'), sex)

Below we want to add a column called `z_scores` to our table, which is the height for age z scores for each child. If missing values are saved as negative numbers, using `compute_z_score` on a height won't work because pygrowup does not compute z-scores for negative heights. Earlier, you got rid of the rows where there was an invalid age, but we will want to keep the rows where a child does not have a height. Use a for loop to get around bad values, inputting `-99` for the z-score if the child has an invalid height.

In [ ]:
z_scores = ... # initialize an array or list to append values to

for i in range(...):
    if ...: # check to see if the height is valid
        ... # make use of the compute_z_score function from above
        ...
    else:
        ... # append the proper value to z_scores if the height is not valid
    
# This line adds a z_scores column to valid_children with the z-scores
valid_children['z_scores'] = z_scores

#After you've added the column, run this code on it.  This changes the data type to a friendlier one for our purposes.
valid_children['z_scores'] = valid_children.apply(float, 'z_scores')

valid_children
### STUDENT ANSWER


Z-scores less than -4 and greater than 4 are implausible.  Replace those observations with -99.

In [ ]:
# use another for loop to check all of the values, then replace the current z_scores column with the updated values
### STUDENT ANSWER


<font color="Blue"> Item 4: Calculate average z-scores for each age (Months Old).  Graph average z-score and age in months.  Do you notice any trends?

*Make sure to not include the -99's!*

In [ ]:
rounded_months = valid_children.apply(round, 'Months Old')
valid_children['Rounded Months'] = rounded_months
max_month = int(max(rounded_months))

averages = ...
for month in range(max_month):
    month_avg = float(np.mean(valid_children.where('z_scores', ...).where('Rounded Months', ...).column('z_scores')))
    ... # append the value to averages

averages_table = Table().with_column('Averages', averages).with_column('Month', range(max_month))
averages_table.plot('Month')
### STUDENT ANSWER


<font color="Blue"> Item 5: What percentage of your population is stunted?  (Height for age z-score less than -2)

*Once again, make sure that you aren't including missing values!*

In [ ]:
### STUDENT ANSWER


<font color="Blue"> Item 6: Calculate average z-scores for boys and girls.  How do these compare?  If there is a difference, do you think these are a result of gender discrimination, or just coincidentally different?  How would you test this?

In [ ]:
valid_children_no_null = valid_children.where('z_scores', ...) # don't include nulls
boy_average = ...
girl_average = ...
(boy_average, girl_average)
### STUDENT ANSWER


Still have time before class ends?  Repeat exercises 4-6 with Weight for Age and/or BMI for age.  What are the differences in these two measures?  What are the implications of these differences?

# Saving the zscores for future joining.
Your Lab 1 data has all the variables we worked with except the variables generated by Pygrowup.  The current data table you have been working with has the zscores and all the variables, but only for chidlren age 5 and under.  Let's save the zscore data in a way that later we can join it to the Lab 1 data.

Make a table that includes household id, individual id and the new variable(s).

In [ ]:
to_save = ... # select the columns we want to save
to_save.show(3)
### STUDENT ANSWER


### Joining
We want to join the tables, linking with household id and individual id.  However, these are currently two separate columns.  We need to combine them into a single Master Id.

We need to assign a unique value to each pair of the form `(<household id>, <individual id>)`. One way to do this is to construct a number of the form `<household id>0<individual_id>`. Here, the 0 acts as a separator, telling us where the household id ends and where the individual id begins. A method to get a number of this form is to use the following equation: $$id = household\_id * 10^n + individual\_id$$ where $n$ is larger than the number of digits in the largest individual id. (Exercise: think about why this is true). 

We have provided the rough code to add a Master ID column. However, you need to fill out the value of n. Use the blank cell below to find out how many digits there are in the largest individual id in your roster.

In [ ]:
### STUDENT ANSWER


In [ ]:
n = ...
to_save['Master ID'] = to_save['Household ID']*n + to_save['Individual ID']
to_save
### STUDENT ANSWER


<font color="Blue"> Item 7: Confirm that there is now a unique id for each observation.  Also, add one more cleaing check to make sure your dataset includes all the children under 5.  Tell me what code you used.

In [ ]:
... # group might be helpful here
### STUDENT ANSWER


In [ ]:
... # check to make sure you have all children under five
### STUDENT ANSWER


Remove the columns Household ID and Indivdiual ID from the table - we don't need them anymore since we have the masterid.

In [ ]:
to_save = ... # drop columns
to_save
### STUDENT ANSWER


Save your data as Lab_3.csv.

In [ ]:
### STUDENT ANSWER


# Backup your notebooks!
Your labs build on each other.  If the server goes down, the will be no way to access your work! Download all your labs to your computer to save a backup.